In [21]:
import numpy as np # linear algebra
import pandas as pd

import numpy as np
import os
import pandas as pd
import cv2
from PIL import Image
import scipy

import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import *
from sklearn.neural_network import MLPClassifier
# import pydot
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from sklearn.model_selection import *
import tensorflow.keras.backend as K

from tqdm import tqdm, tqdm_notebook
from colorama import Fore
import json
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from skimage.io import *
%config Completer.use_jedi = False
import time
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier

from sklearn.metrics import confusion_matrix

import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
from IPython.display import Image, display, Markdown
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from time import perf_counter
import seaborn as sns

def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))

print("All modules have been imported")

All modules have been imported


In [22]:
image_dir = Path('data/colored_images')

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

In [23]:
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Shuffle the DataFrame and reset index
image_df = image_df.sample(frac=1).reset_index(drop = True)

# Show the result
image_df.head()


,Filepath,Label
0,data/colored_images/No_DR/401fdfd0db07.png,No_DR
1,data/colored_images/Moderate/e868c3da340b.png,Moderate
2,data/colored_images/Moderate/1438288bb2e1.png,Moderate
3,data/colored_images/Mild/96a9706b8534.png,Mild
4,data/colored_images/No_DR/7a06ea127e02.png,No_DR


In [24]:
level = []
for i in image_df['Label']:
    if i=='No_DR':
        level.append(0)
    elif i=='Mild':
        level.append(1)
    elif i=='Moderate':
        level.append(2)
    elif i=='Severe':
        level.append(3)
    else:
        level.append(4)

image_df['Level'] = level
image_df.head()

X = []
for i in image_df['Filepath']:
    image = cv2.imread(i)
    X.append(image)
    
X = np.asarray(X)
y = image_df['Level']
Y = np.asarray(y)
        

KeyboardInterrupt: 

In [ ]:
# Y=to_categorical(Y,5)
x_train, x_test1, y_train, y_test1 = train_test_split(X, Y, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test1, y_test1, test_size=0.3, random_state=42)
print(len(x_train),len(x_val),len(x_test))


2563 769 330


In [ ]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(2563, 224, 224, 3)
(769, 224, 224, 3)
(330, 224, 224, 3)


In [ ]:
# Normalize the images
X = X / 255.0

# One-hot encode the labels
y = pd.get_dummies(image_df['Label']).values


In [ ]:
# Assuming you already have y as a one-hot encoded array
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=5)  # 5 classes as per your problem
y_val = to_categorical(y_val, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError

def create_model(activation='relu', loss='categorical_crossentropy'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=(X.shape[1], X.shape[2], X.shape[3])),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation=activation),
        Dropout(0.5),
        Dense(y_train.shape[1], activation='softmax')  # Adjust this to match the number of classes
    ])

    model.compile(optimizer=Adam(), loss=loss, metrics=['accuracy'])
    return model


model = create_model()
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,908,037 (91.20 MB)

 Trainable params: 23,908,037 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model with different activation and loss functions
activations = ['relu', 'tanh']  # Avoid 'leaky_relu' directly unless using advanced layers
losses = ['categorical_crossentropy', 'mean_squared_error']

results = {}
for activation in activations:
    for loss in losses:
        print(f"Training with {activation} activation and {loss} loss")
        model = create_model(activation=activation, loss=loss)
        history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)
        results[(activation, loss)] = history


Training with relu activation and categorical_crossentropy loss
Epoch 1/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 30s 356ms/step - accuracy: 0.5064 - loss: 308.6504 - val_accuracy: 0.6489 - val_loss: 0.9202
Epoch 2/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 28s 350ms/step - accuracy: 0.6168 - loss: 1.0616 - val_accuracy: 0.7061 - val_loss: 0.7995
Epoch 3/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 27s 339ms/step - accuracy: 0.6992 - loss: 0.9203 - val_accuracy: 0.7022 - val_loss: 0.7789
Epoch 4/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 33s 406ms/step - accuracy: 0.6972 - loss: 0.8556 - val_accuracy: 0.7061 - val_loss: 0.7845
Epoch 5/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 35s 432ms/step - accuracy: 0.6885 - loss: 0.8697 - val_accuracy: 0.7113 - val_loss: 0.7613
Epoch 6/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 34s 420ms/step - accuracy: 0.7138 - loss: 0.8209 - val_accuracy: 0.7425 - val_loss: 0.7508
Epoch 7/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 34s 414ms/step - accuracy: 0.7430 - loss: 0.7657 - val_accuracy: 0.7139 - val_loss: 0.7528
Epoch 8/10
81/81 ━━━━━━━━━━━━━━

KeyboardInterrupt: 